# Task 2. I go right through
Imagine a situation: given cities, between some of which roads are laid, and the lengths of these roads are known. It is necessary to find the optimal distances between all pairs of cities - because sometimes it is more profitable to get from one city to another not directly, but through other cities.

The initial data are specified in the form of a matrix A with dimensions $ N \times N $. The $ A_ {ij} $ element is equal to the length of the road between the city $ i $ and the city $ j $, if it exists, and $ -1 $ otherwise.

As a result, the program should return a matrix of the same form, in which the $ A_ {ij} $ element will contain the length of the optimal path between $ i $ and $ j $.

This is a classic graph theory problem solved by Floyd's algorithm. You need to consistently “allow” to pass through the next city, first through the first, then through the second, and so on. Note the similarities between matrix multiplication and this algorithm: you should get pretty much the same code! The difference is that in matrix multiplication you add the products $ A_ {ik} B_ {kj} $, and in Floyd's algorithm you choose the minimum of $ A_ {ij} $ and $ A_ {ik} + A_ {kj} $.

**Result**: In this problem you need to write a program that accepts a two-dimensional matrix $ A $, in which the element $ A_ {ij} $ contains the distance between the cities $ i $ and $ j $ or $ -1 $, if the direct path from $ i $ does not exist in $ j $. The program should output the matrix by the shortest paths between all vertices in the same form.

# Solution

Let the value of $ a_ {ij} ^ k $ be equal to the length of the shortest path from vertex $ i $ to vertex $ j $, and the path can enter intermediate vertices only with numbers less than $ k $ (not counting the beginning and end of the path). That is, $ a_ {ij} ^ 0 $ is the length of the shortest path from $ i $ to $ j $, which does not contain intermediate vertices at all, that is, it consists of only one edge $ ij $, therefore $ a_ {ij} ^ 0 = \omega_ {ij} $. The value $ a_ {ij} ^ 1 = \omega_ {ij} $ is equal to the length of the shortest path that can pass through the intermediate vertex with the number $ 0 $, the path with the weight $ a_ {ij} ^ 2 $ can pass through the intermediate vertices with the numbers $ 0 $ and $ 1 $ and so on. A path with weight $ a_ {ij} ^ N $ can pass through any intermediate vertices, so the value of $ a_ {ij} ^ N $ is equal to the length of the shortest path from $ i $ to $ j $.

Floyd's algorithm sequentially calculates $ a_ {ij} ^ 0 $, $ a_ {ij} ^ 1 $, $ a_ {ij} ^ 2 $, $ \dots $, $ a_ {ij} ^ N $, increasing the value of the parameter $ k $. The initial value is $ a_ {ij} ^ 0 = \omega_ {ij} $.

Now, assuming that the values of $ a_ {ij} ^ {k-1} $ are known, we calculate $ a_ {ij} ^ k $. The shortest path from vertex $ i $ to vertex $ j $ passing through vertices numbered less than $ k $ can either contain or not contain vertex numbered $ k-1 $. 

<img src="Pic.png" width="40%">

If it does not contain the vertex numbered $ k-1 $, then the weight of this path coincides with $ a_ {ij} ^ {k-1} $. If it contains the vertex $ k-1 $, then this path is divided into two parts: $ i - (k-1) $ and $ (k-1) - j $. Each of these parts contains intermediate vertices only with numbers less than $ k-1 $, so the weight of such a path is $ a_ {i (k-1)} ^ {k-1} + a _ {(k-1) j} ^ {k-1} $. Of the two options considered, you must choose the option of the lowest cost, therefore:

$$
a_ {ij} ^ 0 = \min (a_ {ij} ^ {k-1}, a_ {i (k-1)} ^ {k-1} + a _ {(k-1) j} ^ {k- 1})
$$

In this case, the code would look like this:


In [1]:
def floyd(adjacency_matrix, no_path=-1):

    from numpy import array
    from math import inf
  
    adjacency_matrix = array(adjacency_matrix).astype(float)
    adjacency_matrix[adjacency_matrix == no_path] = inf

    N = len(adjacency_matrix)
    A = [[[inf for j in range(N)] for i in range(N)] for k in range(N + 1)]
    for i in range(N):
        for j in range(N):
            A[0][i][j] = adjacency_matrix[i][j]

    for k in range(1, N + 1):
        for i in range(N):
            for j in range(N):
                A[k][i][j] = min(A[k-1][i][j], 
                                A[k-1][i][k-1] + A[k-1][k-1][j])
    return A[-1]

The outer loop in this algorithm sequentially iterates over all vertices, then tries to improve the paths from $ i $ to $ j $, allowing them to pass through the selected vertex. Let's simplify this algorithm, getting rid of the "three-dimensionality" of the array $ A $: we will only store the value of the shortest path from $ i $ to $ j $ in $ A [i] [j] $, and when improving the path, we will write the new length of the path also in $ A [i] [j] $. We will also change the definition of the loop on the variable $ k $, replacing the value of $ k-1 $ with $ k $.

Then the code would look like this:

In [2]:
def floyd(adjacency_matrix, no_path=-1):

    from numpy import array
    from math import inf

    adjacency_matrix = array(adjacency_matrix).astype(float)
    adjacency_matrix[adjacency_matrix == no_path] = inf
  
    N = len(adjacency_matrix)
    for k in range(N):
        for i in range(N):
            for j in range(N):
                adjacency_matrix[i][j] = min(adjacency_matrix[i][j], 
                                            adjacency_matrix[i][k] + adjacency_matrix[k][j])
    return adjacency_matrix

However, if there are edges of negative weight, the values of $ A [i] [j] $ may decrease. Therefore, it may turn out that the value of $ A [i] [j] $ was equal to $ \infty $, and then it decreased due to the presence of edges of negative weight. As a result, the value of $ A [i] [j] $ turned out to be less than $ \infty $ (for example, due to the union of a path of length $ \infty $ and a path of negative weight), but at the same time, the path between the vertices $ i $ and $ j $ no. Therefore, it is necessary either to put additional checks on the fact that $ A [i] [k] $ and $ A [k] [j] $ are not equal to $ \infty $, or values that are slightly less than $ \infty $ are also considered as absence paths.

<img src="Pic2.png" width="30%">

Floyd's algorithm does not work correctly in the presence of a negative weight cycle, but if the path from $ i $ to $ j $ does not contain a negative weight cycle, then the algorithm will find the weight of this path correctly. Also, using this algorithm, you can determine the presence of a cycle of negative weight: if the vertex $ i $ lies on a cycle of negative weight, then the value of $ A [i] [i] $ will be negative after the end of the algorithm.

Thus, we get the final result:

In [3]:
def floyd(adjacency_matrix, no_path=-1, big_num=True):
    '''
    Floyd–Warshall algorithm.
    
    The adjacency matrix is a matrix with 
    dimensions NxN. Element [i,j]
    is equal to the length of the road 
    between city i and city j, 
    if it exists, and -1 otherwise (by default).
    
    Parameters
    ----------
    %(input)s
    adjacency_matrix : array-like
        The adjacency matrix NxN.
    no_path : int, optional
        The no path value between cities.
    big_num : numeric, True (default)
        The number we suppose to be close to inf.
    True means math.inf

    Returns
    -------
    graph : ndarray
        The shortest path matrix.
    '''
    
    from numpy import array
    from math import inf

    if big_num:
        big_num = inf
    
    adjacency_matrix = array(adjacency_matrix).astype(float)
    adjacency_matrix[adjacency_matrix == no_path] = inf
    
    N = len(adjacency_matrix)
    for k in range(N):
        for i in range(N):
            for j in range(N):
                if adjacency_matrix[i][k] < big_num and adjacency_matrix[k][j] < big_num and adjacency_matrix[i,j] > adjacency_matrix[i,k] + adjacency_matrix[k,j]:
                    adjacency_matrix[i,j] = adjacency_matrix[i,k] + adjacency_matrix[k,j]
                    
    return adjacency_matrix

# Examples

As an example we consider the graph

<img src="Graph.png" width="20%">

with adjacency matrix

$
A = \begin{pmatrix}
0 & 100 & -1 & 1\\
100 & 0 & 50 & 3\\
-1 & 50 & 0 & 20\\
1 & 3 & 20 & 0
\end{pmatrix}
$.

In [4]:
A = [[0, 100, -1, 1],[100, 0, 50, 3],[-1, 50, 0, 20],[1, 3, 20, 0]]
A

[[0, 100, -1, 1], [100, 0, 50, 3], [-1, 50, 0, 20], [1, 3, 20, 0]]

Apply Floyd's algorithm to the $A$:

In [5]:
%time
floyd(A)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


array([[ 0.,  4., 21.,  1.],
       [ 4.,  0., 23.,  3.],
       [21., 23.,  0., 20.],
       [ 1.,  3., 20.,  0.]])

And we get the distance matrix:

$
A_{min} = \begin{pmatrix}
0 & 4 & 21 & 1\\
4 & 0 & 23 & 3\\
21 & 23 & 0 & 20\\
1 & 3 & 20 & 0
\end{pmatrix}
$